In [ ]:
import os
import random
import itertools
import re

# 基本的なライブラリ
import numpy as np
import pandas as pd
import scipy as sp
from scipy import stats

# 描画ライブラリ
import matplotlib.pyplot as plt
import seaborn as sns
from seaborn_analyzer import CustomPairPlot
import graphviz
import pydotplus
from IPython.display import Image
from IPython.display import HTML
from six import StringIO
from ipywidgets import interact, FloatSlider

# データセット分割
from sklearn.model_selection import (
    train_test_split,
    StratifiedKFold,
    ShuffleSplit,
    StratifiedShuffleSplit,
)

# 補完
from sklearn.experimental import (
    enable_iterative_imputer,
)  # IterativeImputerをimportするために必要
from sklearn.impute import SimpleImputer, IterativeImputer, KNNImputer

# エンコード
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, OrdinalEncoder

# サンプリング
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

# 特徴量選択
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import (
    GenericUnivariateSelect,
    f_classif,
    mutual_info_classif,
    chi2,
)
from boruta import BorutaPy
from libraries.mrmr import mrmr

# models
from lightgbm import LGBMClassifier
import xgboost as xgb


# 学習中
import optuna
from tqdm import tqdm
from sklearn.model_selection import learning_curve, cross_validate, cross_val_score

# 評価指標
from sklearn.metrics import roc_auc_score
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

import warnings


# config python file
import config

SEED = config.SEED
# INDEX_MICROARRAY = config.INDEX_MICROARRAY
INDEX_MICROARRAY = 2
from functions import *

fix_seed(SEED)


# 最大表示列数の指定（ここでは50列を指定）N
pd.set_option("display.max_columns", 50)
pd.set_option("display.max_rows", 50)

%matplotlib inline

# 目的
遺伝子学的分類に基づいた、予後の2値分類を実施する。  
分類はCLAUDIN_SUBTYPEに基づいて実施。  
予後は5年、10年、15年の3つの年次に分けている。Trueで死亡であることに注意すること。

# データ読み込み
読み込み元：
    config.INTERIM_PREPROCESSED_PROGNOSIS_CROSS_DIR + "/claudin_subtype_chi2"

サブタイプ毎のデータを使用 

In [6]:
X_dict = pickle_load(config.INTERIM_PREPROCESSED_PROGNOSIS_CROSS_DIR + "/X_dict.pickle")
y_dict = pickle_load(config.INTERIM_PREPROCESSED_PROGNOSIS_CROSS_DIR + "/y_dict.pickle")

# 単一グル―プでのモデルのトレーニング

## ベースモデルの学習結果

各特徴量選択されたdfを用いて基本的な2値分類モデルを学習する

In [3]:
def validate_models(mircroarray_type: str, method: str, sampling_type: str = None):
    for year in range(15, 16, 5):  # 予後年数毎のループ
        prognosis_Xlabel = "X{0:0=2}".format(year)
        prognosis_ylabel = "y{0:0=2}".format(year)

        # compare_bcmsは内部でk分割交差検証を行うので、train_test_splitをしなくて良い
        X_train = X_dict[mircroarray_type][method]["train"][prognosis_Xlabel]
        y_train = y_dict[mircroarray_type][method]["train"][prognosis_ylabel]
        X_val = X_dict[mircroarray_type][method]["val"][prognosis_Xlabel]
        y_val = y_dict[mircroarray_type][method]["val"][prognosis_ylabel]

        # accuracyの表示
        print("----------" * 10)
        print("予後年数：{0:0=2}年:".format(year))
        if accuracy_score(y_train, np.zeros(len(y_train))) >= 0.5:
            score = (
                "0>1".format(year),
                round(accuracy_score(y_train, np.zeros(len(y_train))), 3),
            )
        else:
            score = (
                "0>1".format(year),
                round(accuracy_score(y_train, np.ones(len(y_train))), 3),
            )
        print("accuracyベースライン：", score)
        print("使用特徴量：", X_train.columns)
        print("学習サンプルサイズ：", X_train.shape)
        display("ラベル比率：", y_train.value_counts())
        save_dir = (
            config.TABLES_MODELS_PROGNOSIS_CROSS_DIR
            + "/scores_base-model/{0}/{1}".format(mircroarray_type, method)
        )
        make_dir(save_dir)
        sampling = None
        if sampling_type == "sm":
            sampling = SMOTE(random_state=SEED)
        display(
            compare_bcms(
                X_train,
                y_train,
                X_val,
                y_val,
                over_sampling_class=sampling,
                save_path=save_dir + "/{0:0=2}.csv".format(year),
            )
        )

In [4]:
for k in X_dict.keys():
    print(k)
    validate_models(mircroarray_type=k, method="mrmr", sampling_type=None)
    validate_models(mircroarray_type=k, method="mrmr", sampling_type="sm")
    # validate_models(mircroarray_type=k, method="boruta")

mrna_agilent_microarray_zscores_ref_all_samples
----------------------------------------------------------------------------------------------------
予後年数：15年:
accuracyベースライン： ('0>1', 0.533)
使用特徴量： Index(['RACGAP1', 'PFN4', 'NACAP1', 'RAPGEF5', 'EMR3', 'CXorf57', 'DNAJB7',
       'ENC1', 'PCM1', 'KRTAP12-4', 'STAT5B', 'AI659947', 'TOR1B', 'BP432397',
       'RNU11', 'UHRF1', 'RBBP8', 'USP30', 'AK022229', 'FGF13', 'BE858513',
       'HS3ST1', 'DFNB59', 'GAL3ST4', 'AURKA', 'DB341438', 'C9orf95', 'S100P',
       'ANKAR', 'WDR67', 'GABRB1', 'AW572907', 'HIST1H2BF', 'APLN', 'GTPBP5',
       'SPATA4', 'BG218808', 'AW444974', 'FGD3', 'INTS10', 'CBX7', 'BC033399',
       'TSPYL6', 'TMEM26', 'DA697821', 'MST1', 'IGDCC4', 'UBE2C', 'RAB3B'],
      dtype='object')
学習サンプルサイズ： (865, 49)


'ラベル比率：'

1    461
0    404
Name: OS_15years, dtype: int64

11it [00:02,  5.18it/s]


,acc_train,acc_val,f1_train,f1_val
classifier,,,,
Random Forest,0.870520,0.649485,0.881356,0.679245
Sigmoid SVM,0.757225,0.628866,0.781705,0.678571
AdaBoost,0.854335,0.639175,0.865385,0.672897
RBF SVM,0.915607,0.597938,0.920044,0.648649
Logistic Regression,0.806936,0.608247,0.821772,0.648148
Linear SVM,0.810405,0.587629,0.825532,0.636364
Polynomial SVM,0.966474,0.577320,0.968306,0.630631
Naive Bayes,0.736416,0.567010,0.753247,0.596154
Nearest Neighbors,0.800000,0.567010,0.814973,0.596154


...

----------------------------------------------------------------------------------------------------
予後年数：15年:
accuracyベースライン： ('0>1', 0.533)
使用特徴量： Index(['RACGAP1', 'CYP2D6', 'PTX4', 'BIK', 'ENC1', 'DENND6A', 'STAT5B',
       'RNU11', 'DNAJB7', 'PPIL3', 'UHRF1', 'RBBP8', 'MIR137HG', 'GABRB1',
       'TOR1B', 'MST1', 'NMRK1', 'FGF13', 'HIST1H2BF', 'SPATA4', 'TBC1D31',
       'USP30', 'S100P', 'APLN', 'MTG2', 'GAL3ST4', 'FGD3', 'PJVK', 'INTS10',
       'CBX7', 'FCN2', 'RNASE9', 'SPATA18', 'ANKAR', 'SULT4A1', 'TSPYL6',
       'IGDCC4', 'TMEM26', 'PDK3', 'AURKA', 'PRR21', 'DNAAF1', 'GPR151',
       'GASK1A', 'NLRX1', 'RAB3B', 'HS3ST1', 'PKMYT1', 'CATSPERB'],
      dtype='object')
学習サンプルサイズ： (865, 49)


'ラベル比率：'

1    461
0    404
Name: OS_15years, dtype: int64

11it [00:02,  4.99it/s]


,acc_train,acc_val,f1_train,f1_val
classifier,,,,
Quadratic Discriminant Analysis,0.869848,0.690722,0.864253,0.705882
Logistic Regression,0.780911,0.670103,0.781857,0.686275
Decision Tree,0.815618,0.639175,0.818376,0.672897
RBF SVM,0.903471,0.639175,0.901874,0.666667
Sigmoid SVM,0.732104,0.649485,0.730055,0.666667
Linear SVM,0.799349,0.628866,0.801715,0.660377
Random Forest,0.861171,0.628866,0.862069,0.640000
AdaBoost,0.848156,0.618557,0.845815,0.626263
Naive Bayes,0.749458,0.597938,0.748092,0.613861


## lightGBM

In [13]:
X_train = X_dict[config.SET_NAME_MICROARRAY[INDEX_MICROARRAY]]["mrmr"]["train"]["X15"]
y_train = y_dict[config.SET_NAME_MICROARRAY[INDEX_MICROARRAY]]["mrmr"]["train"]["y15"]
X_val = X_dict[config.SET_NAME_MICROARRAY[INDEX_MICROARRAY]]["mrmr"]["val"]["X15"]
y_val = y_dict[config.SET_NAME_MICROARRAY[INDEX_MICROARRAY]]["mrmr"]["val"]["y15"]

params = {
    "max_bin": 100,
    "num_leaves": 30,
    "n_estimators": 800,
    "class_weight": "balanced",
    "random_state": SEED,
}

gbm = LGBMClassifier(**params)
gbm.fit(X_train, y_train)
y_pred = gbm.predict(X_val)
show_scores(y_val, y_pred)

accuracy:  0.6391752577319587
precision:  0.6545454545454545
recall:  0.6923076923076923
f1 score:  0.6728971962616823


## xgboost

In [5]:
import xgboost as xgb

X_train = X_dict[config.SET_NAME_MICROARRAY[INDEX_MICROARRAY]]["mrmr"]["train"]["X15"]
y_train = y_dict[config.SET_NAME_MICROARRAY[INDEX_MICROARRAY]]["mrmr"]["train"]["y15"]
X_val = X_dict[config.SET_NAME_MICROARRAY[INDEX_MICROARRAY]]["mrmr"]["val"]["X15"]
y_val = y_dict[config.SET_NAME_MICROARRAY[INDEX_MICROARRAY]]["mrmr"]["val"]["y15"]

xgb_train = xgb.DMatrix(X_train, label=y_train, feature_names=X_train.columns)
xgb_val = xgb.DMatrix(X_val, label=y_val, feature_names=X_val.columns)


param = {
    # 二値分類問題
    "objective": "binary:logistic",
}
clf = xgb.train(param, xgb_train)
y_pred_proba = clf.predict(xgb_val)
y_pred = np.where(y_pred_proba > 0.5, 1, 0)

accuracy_score(y_val, y_pred), f1_score(y_val, y_pred)

[05:23:04] WARNING: /tmp/abs_40obctay9q/croots/recipe/xgboost-split_1659548945886/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


(0.6494845360824743, 0.6730769230769231)

## モデル毎の比較（年数など）

### 予測モデル、特徴量数、年数毎の比較

In [5]:
def model2result_list(
    clf: callable,
    X_train: pd.DataFrame(),
    y_train: pd.DataFrame(),
    X_test: pd.DataFrame(),
    y_test: pd.DataFrame(),
    model_name: str,
    year: int,
    k: int,
) -> list():
    clf.fit(X_train, y_train)
    pred = clf.predict(X_test)
    acc = accuracy_score(y_test, pred)
    pre = precision_score(y_test, pred)
    rec = recall_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    return [year, model_name, acc, pre, rec, f1, k]

In [6]:
row = []
fold = 5
for year in tqdm(range(15, 16, 5)):  # 予後年数毎のループ
    prognosis_Xlabel = "X{0:0=2}".format(year)
    prognosis_ylabel = "y{0:0=2}".format(year)
    for k in range(10, 51, 10):
        tmp = None
        X = X_dict["mrna_agilent_microarray_zscores_ref_diploid_samples"]["mrmr"][
            "train"
        ][prognosis_Xlabel]
        y = y_dict["mrna_agilent_microarray_zscores_ref_diploid_samples"]["mrmr"][
            "train"
        ][prognosis_ylabel]

        skf = StratifiedKFold(n_splits=fold, shuffle=True, random_state=SEED)
        for train_index, val_indes in skf.split(X, y):
            X_train, X_val = X.iloc[train_index], X.iloc[val_indes]
            y_train, y_val = y.iloc[train_index], y.iloc[val_indes]

            assert X_train.shape[0] == y_train.shape[0], "train size is incorrect"
            assert X_val.shape[0] == y_val.shape[0], "test size is incorrect"

            # 特徴量選択の実施（mRMR）
            features = mrmr.mrmr_classif(
                X=X_train,
                y=y_train,
                K=k,
                show_progress=False,
            )

            # 特徴量選択で選ばれた特徴量の抽出（訓練、テストデータに適用）
            # list変換するのはfuture warningが出るため
            X_train = X_train[features]
            X_val = X_val[features]

            # スケーリングの実施
            X_train, X_val = transform_std(X_train, X_val)
            # X_train, X_val = transform_norm(X_train, X_val)

            # サンプリングの実装
            # positive_count_train = y_train.value_counts().sort_values()[0]
            rus = RandomUnderSampler(sampling_strategy="all", random_state=SEED)
            # smote = SMOTE(sampling_strategy="all", random_state=SEED)
            X_resampled, y_resampled = rus.fit_resample(X_train, y_train)

            # accuracyの表示
            """
            print("予後年数：{0:0=2}年:".format(year))
            print("学習サンプルサイズ：", X_train.shape)
            display("ラベル比率：", y_train.value_counts())        
            """

            # ここからモデルのtrain、x軸をkとしたacc, f1の推移plot
            # print("Random Forest")
            params = {
                "n_estimators": 500,
                "criterion": "entropy",
                "max_depth": 5,
                "max_leaf_nodes": 20,
                "max_features": "log2",
                "class_weight": "balanced",
                "random_state": SEED,
            }
            rf = RandomForestClassifier(**params)
            tmp = model2result_list(rf, X_train, y_train, X_val, y_val, "rf", year, k)
            row.append(tmp)

            # print("Logistic Regression")
            params = {
                "penalty": "l2",
                "C": 0.5,
                "solver": "saga",
                "class_weight": "balanced",
            }
            lr = LogisticRegression(**params)
            tmp = model2result_list(lr, X_train, y_train, X_val, y_val, "lr", year, k)
            row.append(tmp)

            # print("Support Vector Machine")
            params = {
                "C": 50,
                "kernel": "sigmoid",
                "decision_function_shape": "ovr",
                "class_weight": "balanced",
            }
            svm = SVC(**params)
            tmp = model2result_list(svm, X_train, y_train, X_val, y_val, "svm", year, k)
            row.append(tmp)

    # 結果をdfにまとめている
    df_result = pd.DataFrame(
        row,
        columns=[
            "year",
            "model_name",
            "acc",
            "precision",
            "recall",
            "f1",
            "feature size",
        ],
    )

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [02:02<00:00, 122.37s/it]


In [7]:
df_result[df_result["model_name"] == "rf"].groupby(
    ["year", "feature size"]
).mean().sort_values("acc", ascending=False)

acc  precision    recall        f1
year feature size                                         
15   40            0.720231   0.734554  0.746166  0.739993
     50            0.720231   0.734554  0.746166  0.739993
     30            0.713295   0.725555  0.743922  0.734229
     20            0.683237   0.694121  0.728822  0.710235
     10            0.677457   0.695627  0.704979  0.699879

In [8]:
df_result[df_result["model_name"] == "lr"].groupby(
    ["year", "feature size"]
).mean().sort_values("acc", ascending=False)

acc  precision    recall        f1
year feature size                                         
15   40            0.732948   0.756673  0.737564  0.746544
     50            0.732948   0.756673  0.737564  0.746544
     30            0.715607   0.739345  0.722394  0.730340
     20            0.695954   0.725919  0.689808  0.707205
     10            0.690173   0.722748  0.681043  0.701143

In [9]:
df_result[df_result["model_name"] == "svm"].groupby(
    ["year", "feature size"]
).mean().sort_values("acc", ascending=False)

acc  precision    recall        f1
year feature size                                         
15   40            0.680925   0.712835  0.674638  0.692703
     50            0.680925   0.712835  0.674638  0.692703
     20            0.669364   0.697698  0.670266  0.683482
     30            0.669364   0.700168  0.670196  0.683615
     10            0.611561   0.653488  0.583567  0.615024

# subtype毎のベースライン・学習

In [3]:
def validate_models_subtype(
    mircroarray_type: str,
    method: str,
):
    subtypes = [
        "claudin-low",
        "LumA",
        "LumB",
        "Her2",
        "Normal",
        "Basal",
    ]

    for year in range(15, 16, 5):  # 予後年数毎のループ
        print("====={0:0=2}".format(year) * 10)

        for subtype in subtypes:
            prognosis_Xlabel = "X{0:0=2}_{1}".format(year, subtype)
            prognosis_ylabel = "y{0:0=2}_{1}".format(year, subtype)
            X_train = X_dict[mircroarray_type]["claudin_subtype"][method]["train"][
                prognosis_Xlabel
            ]
            y_train = y_dict[mircroarray_type]["claudin_subtype"][method]["train"][
                prognosis_ylabel
            ]
            X_val = X_dict[mircroarray_type]["claudin_subtype"][method]["val"][
                prognosis_Xlabel
            ]
            y_val = y_dict[mircroarray_type]["claudin_subtype"][method]["val"][
                prognosis_ylabel
            ]
            assert X_train.shape[0] == y_train.shape[0], "train size is incorrect"
            assert X_val.shape[0] == y_val.shape[0], "val size is incorrect"
            if X_train.shape[1] == 0:
                print(
                    "**--WARNING: FEATURE NUM is 0! -> subtype {0} train is skipped!--**".format(
                        subtype
                    )
                )
                continue

            # accuracyの表示
            print("----------" * 10)
            print("subtype: ", subtype)
            print("予後年数：{0:0=2}年:".format(year))
            if accuracy_score(y_train, np.zeros(len(y_train))) >= 0.5:
                score = (
                    "0>1".format(year),
                    round(accuracy_score(y_train, np.zeros(len(y_train))), 3),
                )
            else:
                score = (
                    "0>1".format(year),
                    round(accuracy_score(y_train, np.ones(len(y_train))), 3),
                )
            print("accuracyベースライン：", score)
            print("使用特徴量：", X_train.columns)
            print("学習サンプルサイズ：", X_train.shape)
            display("ラベル比率：", y_train.value_counts())

            sm = SMOTE(random_state=SEED)
            display(
                compare_bcms(
                    X_train,
                    y_train,
                    X_val,
                    y_val,
                    over_sampling_class=sm,
                )
            )

In [7]:
for k in X_dict.keys():
    print(k)
    validate_models_subtype(mircroarray_type=k, method="mrmr")

mrna_agilent_microarray_zscores_ref_all_samples
=====15=====15=====15=====15=====15=====15=====15=====15=====15=====15
----------------------------------------------------------------------------------------------------
subtype:  claudin-low
予後年数：15年:
accuracyベースライン： ('0>1', 0.562)
使用特徴量： Index(['BF511322', 'BIRC7', 'ZFP90', 'CHRNB3', 'IL34', 'AA939346', 'AW301012',
       'DB341932', 'MRPL53', 'CHDH', 'DA102929', 'GTPBP5', 'BF447974', 'NAV2',
       'C20orf29', 'NCRNA00160', 'PRPSAP1', 'CD511953', 'STX1B', 'COBL'],
      dtype='object')
学習サンプルサイズ： (89, 20)


'ラベル比率：'

0    50
1    39
Name: OS_15years, dtype: int64

11it [00:00, 41.97it/s]


,acc_train,acc_val,f1_train,f1_val
classifier,,,,
Decision Tree,0.83,0.6,0.852174,0.600000
Quadratic Discriminant Analysis,1.00,0.6,1.000000,0.500000
Linear SVM,1.00,0.5,1.000000,0.285714
Logistic Regression,1.00,0.4,1.000000,0.250000
Polynomial SVM,1.00,0.4,1.000000,0.250000
RBF SVM,1.00,0.4,1.000000,0.250000
Sigmoid SVM,0.95,0.4,0.951456,0.250000
Nearest Neighbors,0.95,0.3,0.952381,0.222222
AdaBoost,1.00,0.2,1.000000,0.000000


...

----------------------------------------------------------------------------------------------------
subtype:  Basal
予後年数：15年:
accuracyベースライン： ('0>1', 0.61)
使用特徴量： Index(['FBXO31', 'OR2T29', 'OR2AG2', 'ALAS2', 'KLHL4', 'CAMK1D', 'YWHAEP7',
       'PTX4', 'KRTAP5-5', 'OR52N4', 'STATH', 'NUDT12'],
      dtype='object')
学習サンプルサイズ： (105, 12)


'ラベル比率：'

1    64
0    41
Name: OS_15years, dtype: int64

11it [00:00, 49.39it/s]


,acc_train,acc_val,f1_train,f1_val
classifier,,,,
Decision Tree,0.828125,0.750000,0.822581,0.769231
Linear SVM,0.867188,0.666667,0.866142,0.750000
Logistic Regression,0.867188,0.666667,0.868217,0.750000
Sigmoid SVM,0.835938,0.583333,0.839695,0.705882
AdaBoost,1.000000,0.583333,1.000000,0.666667
Random Forest,0.945312,0.583333,0.944882,0.666667
RBF SVM,0.968750,0.500000,0.968254,0.625000
Naive Bayes,0.859375,0.416667,0.867647,0.588235
Polynomial SVM,0.953125,0.416667,0.950820,0.533333
